In [1]:
import logging
log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 15
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

import os
from dotenv import load_dotenv
load_dotenv()
secret_name = "OPENAI_API_KEY_CEMAD"
openai_api_key = os.getenv(secret_name)
key = os.getenv('DECRYPTION_KEY_CEMAD')


In [2]:
import os
import importlib

from openai import OpenAI

from regulations_rag.file_tools import load_parquet_data, save_parquet_data


import data_tools.index_tools
importlib.reload(data_tools.index_tools)
from data_tools.index_tools import update_text_in_index, add_to_index, remove_from_index

openai_client = OpenAI(api_key=openai_api_key,)
key = os.getenv('excon_encryption_key')

#data  = CEMADData(key)

In [3]:
#to_change = "index"
to_change = "index_plus"
if to_change == "index":
    file = "./inputs/index/ad_index.parquet"
else:
    file = "./inputs/index/ad_index_plus.parquet"

doc_index = load_parquet_data(file, key)

embedding_model =  "text-embedding-3-large"
embedding_dimensions = 1024


In [4]:
doc_index[doc_index["section_reference"] == "B.16(E)"]
# tmp = doc_index[doc_index["section_reference"] == "B.16(E)"]
# for index, row in tmp.iterrows():
#     print("--" + row['text'])

,section_reference,text,source,embedding,document
180,B.16(E),Credit and/or debit cards. Miscellaneous payme...,heading,"[-0.02003210037946701, 0.0697329193353653, -0....",CEMAD
622,B.16(E),Resident individuals or local entities with ba...,summary,"[-0.05386892333626747, 0.05558310076594353, -0...",CEMAD
1191,B.16(E),Can South African residents use their bank car...,question,"[-0.03829977661371231, 0.07157550752162933, -0...",CEMAD
1192,B.16(E),What is the limit for each transaction when us...,question,"[-0.037802569568157196, 0.02708139829337597, -...",CEMAD
1193,B.16(E),Can credit cards be used to fund international...,question,"[-0.09016311168670654, -0.006297208834439516, ...",CEMAD
1501,B.16(E),Can I use my credit card for online purchases?,question,"[-0.06440070271492004, 0.03733660280704498, -0...",CEMAD


In [30]:
doc_index.at[1501, "document"] = "CEMAD"

## Change an existing entry

## Change an entry

In [84]:
text_to_change = "What does cemad say about commodities?"
changed_text = "commodities"
document = "CEMAD"
doc_index = update_text_in_index(openai_client = openai_client, index_df = doc_index , text_to_change=text_to_change, changed_text=changed_text, embedding_model=embedding_model, embedding_dimensions=embedding_dimensions, document = document)

# SAVE IT
# save_parquet_data(doc_index, file, key)

## Add an entry

In [24]:
text_to_add = "Can I use my credit card for online purchases?"
section_reference = "B.16(E)"
source = "question"
doc_index = add_to_index(openai_client = openai_client, index_df = doc_index, text = text_to_add, section_reference = section_reference, source = source, embedding_model = embedding_model, embedding_dimensions= embedding_dimensions)

# new_embedding = get_ada_embedding(openai_client = openai_client, text = text, model = embedding_model, dimensions = embedding_dimensions)
# new_df = pd.DataFrame([[section_reference, text, source, new_embedding]],columns = ["section_reference", "text", "source" , "embedding"])
# index_df = pd.concat([index_df, new_df], ignore_index=True)
# return index_df


# SAVE IT
# save_parquet_data(doc_index, file, key)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [32]:
save_parquet_data(doc_index, file, key)

## Delete an entry

In [92]:
text_to_delete = "more commodities"
doc_index = remove_from_index(index_df = doc_index, text_to_delete = text_to_delete)

# SAVE IT
# save_parquet_data(doc_index, file, key)

In [36]:
definitions = load_parquet_data("./inputs/index/ad_definitions.parquet")
definitions

,definition,source,embedding,document,section_reference
0,"Treasury means, in relation to any matter cont...",all,"[0.012373767793178558, 0.03887529298663139, -0...",CEMAD,A.1
1,ADLA Manual means the Currency and Exchanges M...,all,"[0.018092811107635498, 0.010141296312212944, -...",CEMAD,A.1
2,"Affected person means a body corporate, founda...",all,"[-0.037813764065504074, 0.07012207806110382, -...",CEMAD,A.1
3,Africa means any country forming part of the A...,all,"[0.015041325241327286, 0.10257036238908768, -0...",CEMAD,A.1
4,"Authorised Dealer means, in relation to any tr...",all,"[0.02330564334988594, 0.032766155898571014, -0...",CEMAD,A.1
...,...,...,...,...,...
97,Local outsourcing is the subcontracting of bus...,cloud,"[-0.0006997798918746412, 0.012086263857781887,...",CEMAD,J.(D)(i)
98,International outsourcing is the subcontractin...,cloud,"[-0.00498924357816577, 0.01612837053835392, -0...",CEMAD,J.(D)(i)
99,Regulatory access to data refers to FinSurv's ...,cloud,"[-0.032946694642305374, 0.11769585311412811, -...",CEMAD,J.(D)(i)
100,System replication refers to duplication of tr...,cloud,"[0.0027270971331745386, 0.013970836065709591, ...",CEMAD,J.(D)(i)


In [37]:
from regulations_rag.embeddings import get_ada_embedding
from regulations_rag.embeddings import get_closest_nodes

user_content = "Can I open a non-resident account for a CMA resident from another CMA country?"
question_embedding = get_ada_embedding(openai_client = openai_client, text = user_content, model = embedding_model, dimensions = embedding_dimensions)
relevant_definitions = get_closest_nodes(definitions, embedding_column_name="embedding", content_embedding=question_embedding, threshold=0.9)
relevant_definitions


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


,definition,source,embedding,document,section_reference,cosine_distance
44,Resident account means the account of a person...,all,"[-0.0138019984588027, 0.06224393844604492, -0....",CEMAD,A.1,0.410117
34,Non-resident means a person (i.e. a natural pe...,all,"[-0.008788907900452614, -0.0058042630553245544...",CEMAD,A.1,0.438435
35,Non-resident area means all countries other th...,all,"[0.0033106659539043903, -0.024005355313420296,...",CEMAD,A.1,0.440496
22,Foreign bank account means a foreign currency ...,all,"[0.021879395470023155, 0.035312242805957794, -...",CEMAD,A.1,0.490314
24,Foreign currency account means a foreign curre...,all,"[0.007825705222785473, 0.0609523244202137, -0....",CEMAD,A.1,0.531112
...,...,...,...,...,...,...
70,(xvi) Short-term Insurance Act \n ...,insurance,"[0.04359523579478264, 0.020466331392526627, 0....",CEMAD,B.10.(B),0.875374
96,Cloud computing is defined as a model for enab...,cloud,"[-0.07864412665367126, 0.014763019979000092, -...",CEMAD,J.(D)(i),0.882673
38,"Pension Funds Act means Pension Funds Act, 195...",all,"[-8.884889393812045e-05, 0.0902838483452797, -...",CEMAD,A.1,0.884064
39,Rand means the monetary unit of South Africa a...,all,"[-0.013166881166398525, 0.013540474697947502, ...",CEMAD,A.1,0.888295
